In [1]:
import requests
import time
import sys
import pandas as pd
import os

In [2]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install --upgrade google-cloud-storage

    100% |████████████████████████████████| 81kB 4.2MB/s ta 0:00:01
  Found existing installation: google-resumable-media 0.4.1
    Uninstalling google-resumable-media-0.4.1:
      Successfully uninstalled google-resumable-media-0.4.1
  Found existing installation: google-cloud-storage 1.22.0
    Uninstalling google-cloud-storage-1.22.0:
      Successfully uninstalled google-cloud-storage-1.22.0


In [4]:
%env GOOGLE_APPLICATION_CREDENTIALS=/Users/willyraedy/Sync/SideProjects/RedditResearch/credentials-ebeb319739c4.json

env: GOOGLE_APPLICATION_CREDENTIALS=/Users/willyraedy/Sync/SideProjects/RedditResearch/credentials-ebeb319739c4.json


In [5]:
# Imports the Google Cloud client library
from google.cloud import storage

# Instantiates a client
storage_client = storage.Client()

In [6]:
# setup
UPLOAD_SIZE = 10000
IN_NETWORK_STRING = 'latinoamerica' # 'AskReddit'
LOGGING = True

In [37]:
def fetch_page(param, value):
    resp = requests.get(f'https://www.reddit.com/reddits.json?limit=100&{param}={value}&count=555', headers={'User-Agent': 'script:melis-thesis:v0.0.1 (by /u/wilburRay)'})
    data = resp.json()
    new_pagination_value = data['data'][param]
    number_returned = data['data']['dist']
    new_reddits = list(map(lambda r: r['data'], data['data']['children']))
    return dict(new_pagination_value=new_pagination_value, new_reddits=new_reddits, number_returned=number_returned)

In [38]:
def upload_file(dataframe, gcp_filename):
    temp_filename = 'tempCSVToUpload.csv'
    dataframe.to_csv(temp_filename)
    
    bucket = storage_client.get_bucket('meli_thesis')
    blob = bucket.blob(gcp_filename)
    blob.upload_from_filename(temp_filename)
    
    os.remove(temp_filename)

In [39]:
def filter_network_reddits(dataframe):
    return dataframe[dataframe['description_html'].str.contains(IN_NETWORK_STRING, na=False)]

In [40]:
def process_chunk(list_of_reddits):
    # upload raw results
    
    df = pd.DataFrame(list_of_reddits)

    from_id = list_of_reddits[0]['id']
    to_id = list_of_reddits[-1]['id']
    count = len(list_of_reddits)
    raw_gcp_filename = f'raw/{from_id}-{to_id}-reddits-{count}'
    
    if LOGGING:
        print(f'raw_gcp_filename: {raw_gcp_filename}')
        print(f'raw_head: {df.head(5)}')
    
    if not LOGGING:
        upload_file(df, raw_gcp_filename)
    
    # filter out in network reddits
    # upload in network reddits
    
    in_network = filter_network_reddits(df)
    
    in_network_count = in_network.shape[0]
    network_gcp_filename = f'network/{from_id}-{to_id}-reddits-{in_network_count}'    
    
    if LOGGING:
        print(f'in_network_filename: {network_gcp_filename}')
        print(f'network_head: {in_network.head(5)}')

    if not LOGGING:
        upload_file(in_network, network_gcp_filename)
    

In [43]:
def fetch_all():
    returned = 100
    after = ''
    before = ''
    all_reddits = []
    total_scraped = 0
    
#     while after != None:
#         if LOGGING:
#             print('###### new batch #####')
#         try:
#             result = fetch_page('after', after)
#             all_reddits += list(map(lambda r: dict(id=r['id'], display_name=r['display_name'], public_description=r['public_description'], description_html=r['description_html'], subscribers=r['subscribers']), result['new_reddits']))
#             after = result['new_pagination_value']
#             returned = result['number_returned']
#             total_scraped += returned
#             time.sleep(1)
            
#             if LOGGING:
#                 print(f'after: {after}')
#                 print(pd.DataFrame(all_reddits).tail(3))
#                 print(f'total scraped: {total_scraped}')

#             if len(all_reddits) % UPLOAD_SIZE == 0:
#                 print('***** uploading data *****')
#                 process_chunk(all_reddits)
#                 all_reddits = []
#         except KeyboardInterrupt:
#             raise
#         except:
#             e = sys.exc_info()[0]
#             print(e)
#             if LOGGING:
#                 raise(e)
    
    print('Switching to Before')
    
    while before != None:
        if LOGGING:
            print('###### new batch #####')
        try:
            result = fetch_page('before', before)
            all_reddits += list(map(lambda r: dict(id=r['id'], display_name=r['display_name'], public_description=r['public_description'], description_html=r['description_html'], subscribers=r['subscribers']), result['new_reddits']))
            before = result['new_pagination_value']
            returned = result['number_returned']
            total_scraped += returned
            time.sleep(1)

            if LOGGING:
                print(f'before: {before}')
                print(pd.DataFrame(all_reddits).tail(3))
                print(f'total scraped: {total_scraped}')

            if len(all_reddits) % UPLOAD_SIZE == 0:
                print('***** uploading data *****')
                process_chunk(all_reddits)
                all_reddits = []
        except KeyboardInterrupt:
            raise
        except:
            e = sys.exc_info()[0]
            print(e)
            if LOGGING:
                raise(e)
                
    if len(all_reddits) > 0:
        print('processing last chunk')
        process_chunk(all_reddits)
    print('done')
    return all_reddits

In [44]:
all_reddits = fetch_all()

Switching to Before
###### new batch #####
before: t5_2qh3l
                                     description_html display_name     id  \
97  &lt;!-- SC_OFF --&gt;&lt;div class="md"&gt;&lt...          CFB  2qm9d   
98  &lt;!-- SC_OFF --&gt;&lt;div class="md"&gt;&lt...     facepalm  2r5rp   
99  &lt;!-- SC_OFF --&gt;&lt;div class="md"&gt;&lt...        space  2qh87   

                                   public_description  subscribers  
97  A forum for all things college football. Prima...       671152  
98  A subreddit for you to share the stupidity of ...      3027608  
99  Share &amp; discuss informative content on:\n\...     16020450  
total scraped: 100
###### new batch #####
before: None
                                     description_html display_name     id  \
97  &lt;!-- SC_OFF --&gt;&lt;div class="md"&gt;&lt...          CFB  2qm9d   
98  &lt;!-- SC_OFF --&gt;&lt;div class="md"&gt;&lt...     facepalm  2r5rp   
99  &lt;!-- SC_OFF --&gt;&lt;div class="md"&gt;&lt...        space  2

In [86]:
df = pd.DataFrame(all_reddits)

In [87]:
df.shape

(3673, 5)

In [107]:
mex = df[df['display_name'] == 'guadalajara']

In [108]:
mex.head()

,description_html,display_name,id,public_description,subscribers


In [103]:
d = mex.to_dict()

In [104]:
d

{'description_html': {1161: '&lt;!-- SC_OFF --&gt;&lt;div class="md"&gt;&lt;p&gt;&lt;a href="http://nm.reddit.com/r/Mexico/#nm"&gt;MODO NOCTURNO&lt;/a&gt;\n&lt;a href="http://reddit.com/r/Mexico/#dm"&gt;NORMAL&lt;/a&gt;&lt;/p&gt;\n\n&lt;h3&gt;Descripción&lt;/h3&gt;\n\n&lt;p&gt;Noticias, imágenes y links relacionados con México son bienvenidos, así como anuncios y preguntas dirigidas a la comunidad.&lt;/p&gt;\n\n&lt;p&gt;News, images and links related to Mexico are welcome, as well as announcements and questions addressed to the community.&lt;/p&gt;\n\n&lt;h3&gt;Atención&lt;/h3&gt;\n\n&lt;ul&gt;\n&lt;li&gt;&lt;strong&gt;NO SPAM&lt;/strong&gt;&lt;/li&gt;\n&lt;li&gt;&lt;strong&gt;NO EDITES TÍTULOS / DON&amp;#39;T EDIT TITLES&lt;/strong&gt;&lt;/li&gt;\n&lt;li&gt;&lt;strong&gt;NO ATAQUES PERSONALES / NO PERSONAL ATTACKS&lt;/strong&gt;&lt;/li&gt;\n&lt;li&gt;&lt;strong&gt;NO INSULTES&lt;/strong&gt;&lt;/li&gt;\n&lt;/ul&gt;\n\n&lt;p&gt;Por favor lee el &lt;a href="http://www.reddit.com/r/mexico